In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ayarkhan01/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Load stop words and convert to list
stop_words = list(stopwords.words('english'))

# Create a CountVectorizer object with stop words
vectorizer = CountVectorizer(stop_words=stop_words)

In [22]:
from joblib import load

model = load('multinomial_nb_model.joblib')


/home/ayarkhan01/Desktop/Projects/Project 2/.venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.5.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
data_imported= pd.read_csv('tweets.csv')

In [24]:
data_imported

,User ID,Tweet,Date
0,761001004091400192,🚨BREAKING🚨\n\nAPPLE $AAPL GETS A PRICE RAISE F...,2024-06-20 10:25:55+00:00
1,1089978712685273090,Why Apple's Strategy for Artificial Intelligen...,2024-06-20 10:21:33+00:00
2,1669688122064420866,شركة أميانتيت العربية السعودية (https://t.co/x...,2024-06-20 10:20:38+00:00
3,120140442,Why Apple's Strategy for Artificial Intelligen...,2024-06-20 10:13:05+00:00
4,1492239884672765954,“ I don’t like the stock market because capita...,2024-06-20 10:09:14+00:00
...,...,...,...
95,1601683238,as NY there to keep up with goings on World Mo...,2024-06-19 22:26:01+00:00
96,174853,"@ID_AA_Carmack See also, bought a lot of Apple...",2024-06-19 22:25:26+00:00
97,1798974061558616064,"🚨BREAKING: @nvidia , a leader in AI chips and ...",2024-06-19 22:19:05+00:00
98,1242781069,Market summary on $APPLE stock as of today. ht...,2024-06-19 22:18:45+00:00


In [25]:
data = data_imported[['Tweet']].copy()
data.columns = ['sentence']

In [26]:
data

,sentence
0,🚨BREAKING🚨\n\nAPPLE $AAPL GETS A PRICE RAISE F...
1,Why Apple's Strategy for Artificial Intelligen...
2,شركة أميانتيت العربية السعودية (https://t.co/x...
3,Why Apple's Strategy for Artificial Intelligen...
4,“ I don’t like the stock market because capita...
...,...
95,as NY there to keep up with goings on World Mo...
96,"@ID_AA_Carmack See also, bought a lot of Apple..."
97,"🚨BREAKING: @nvidia , a leader in AI chips and ..."
98,Market summary on $APPLE stock as of today. ht...


In [27]:
from joblib import load

vectorizer = load('vectorizer.joblib')


/home/ayarkhan01/Desktop/Projects/Project 2/.venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.5.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [28]:
predicted_sentiments = []

# Iterate over each sentence in the DataFrame
for sentence in data['sentence']:
    # Transform the sentence to vector
    sentence_vector = vectorizer.transform([sentence])
    # Predict sentiment
    predicted_sentiment = model.predict(sentence_vector)
    # Append the predicted sentiment to the list
    predicted_sentiments.append(predicted_sentiment[0])

# Add the predictions as a new column to the DataFrame
data['predicted_sentiment'] = predicted_sentiments

In [29]:
html_table = data.to_html()

In [30]:
# Count the number of positive and negative sentiments
sentiment_counts = data['predicted_sentiment'].value_counts()
html_sentiment_counts = sentiment_counts.to_frame().to_html()

if 'Positive' in sentiment_counts and 'Negative' in sentiment_counts:
    if sentiment_counts['Positive'] > sentiment_counts['Negative']:
        overall_sentiment = 'Positive'
    elif sentiment_counts['Positive'] == sentiment_counts['Negative']:
        overall_sentiment = 'Neutral'
    else:
        overall_sentiment = 'Negative'
elif 'Positive' in sentiment_counts:
    overall_sentiment = 'Positive'
else:
    overall_sentiment = 'Negative'

print(f"The overall sentiment for the day is: {overall_sentiment}")




The overall sentiment for the day is: Positive


In [31]:
def color_sentiment(val):
    color = 'green' if val == 'Positive' else 'red'
    return f'color: {color}'

In [32]:
styled_data = data.style.applymap(color_sentiment, subset=['predicted_sentiment'])

/tmp/ipykernel_332/1255149020.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_data = data.style.applymap(color_sentiment, subset=['predicted_sentiment'])


In [33]:
html_data = styled_data.to_html()

In [34]:
# Determine the background color based on sentiment
bg_color = 'green' if overall_sentiment == 'Positive' else 'red'

In [35]:
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Analysis Results</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background: #f4f4f4;
            margin: 40px;
            color: #333;
        }}
        table {{
            border-collapse: collapse;
            width: 100%;
            margin-bottom: 20px;
        }}
        th, td {{
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }}
        th {{
            background-color: #3b81e2;
            color: white;
        }}
        tr:nth-child(even){{background-color: #f2f2f2}}
        .sentiment-header {{
            background-color: {bg_color};  /* Dynamic background color */
            color: white; /* White text for better contrast */
            padding: 10px;
        }}
    </style>
</head>
<body>
    <h1>Analysis Results</h1>
    <h2>Sentiment Counts</h2>
    {html_sentiment_counts}
    <h2 class='sentiment-header'>Overall Sentiment for the Day: {overall_sentiment}</h2>  <!-- Styled based on sentiment -->
    <h2>Data Table</h2>
    {html_data}  <!-- Correctly include the styled data table -->
</body>
</html>
"""

# You can now save or display this HTML content


In [36]:

# Save the HTML content to a file
with open('output.html', 'w') as f:
    f.write(html_content)

# Print a confirmation
print("HTML file has been created and saved as 'output.html'.")


HTML file has been created and saved as 'output.html'.
